# ML example: predict order total

This notebook trains a simple regression model to predict order `total_amount` from `item_count` and other lightweight features.
It will: 
- try to read data from Postgres if `DATABASE_URL` is set, otherwise fall back to `data/processed/orders_processed.csv`.
- train a simple scikit-learn model and evaluate it.
- save the trained model to `models/order_total_model.pkl`.

In [1]:
# Imports
import os
import pandas as pd
from pathlib import Path

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Optional: SQLAlchemy for DB read
try:
    from sqlalchemy import create_engine
    SQLA_AVAILABLE = True
except Exception:
    SQLA_AVAILABLE = False

In [4]:
# Load data: prefer DATABASE_URL, else processed CSV, else raw Olist files
import os
from pathlib import Path
import pandas as pd

# Try database first
db_url = os.environ.get('DATABASE_URL')
df = None
if db_url:
    try:
        from sqlalchemy import create_engine
        engine = create_engine(db_url)
        for table in ['orders', 'fact_orders']:
            try:
                df = pd.read_sql_table(table, con=engine)
                print(f'Loaded data from database table: {table}')
                break
            except Exception:
                continue
    except Exception as e:
        print('DB read failed, falling back to files:', e)

# Try processed files
if df is None:
    candidates = [
        Path('data/processed/orders_processed.csv'),
        Path('data/staging/clean/orders_clean.csv'),
        Path('../data/processed/orders_processed.csv'),
        Path('../data/staging/clean/orders_clean.csv')
    ]
    for p in candidates:
        if p.exists():
            df = pd.read_csv(p)
            print('Loaded processed data from', p)
            break

# Fallback: build minimal features from raw Olist datasets
if df is None:
    raw_candidates = [
        Path('brazilian dataset'),
        Path('../brazilian dataset'),
        Path('../../brazilian dataset')
    ]
    raw_dir = next((p for p in raw_candidates if p.exists()), None)
    if raw_dir is None:
        raise FileNotFoundError("Could not locate 'brazilian dataset' directory from notebook location.")

    orders_p = raw_dir / 'olist_orders_dataset.csv'
    items_p = raw_dir / 'olist_order_items_dataset.csv'
    payments_p = raw_dir / 'olist_order_payments_dataset.csv'
    if not (orders_p.exists() and items_p.exists()):
        raise FileNotFoundError(f"Expected raw Olist files in '{raw_dir}'")

    orders = pd.read_csv(orders_p, parse_dates=['order_purchase_timestamp'])
    items = pd.read_csv(items_p)

    items_agg = items.groupby('order_id').agg(
        items_count=('order_item_id', 'count'),
        total_price=('price', 'sum'),
        total_freight=('freight_value', 'sum'),
        avg_item_price=('price', 'mean')
    ).reset_index()

    df = orders.merge(items_agg, on='order_id', how='left')

    if payments_p.exists():
        pays = pd.read_csv(payments_p)
        pays_agg = pays.groupby('order_id').agg(
            payment_value=('payment_value', 'sum'),
            payment_installments=('payment_installments', 'max')
        ).reset_index()
        df = df.merge(pays_agg, on='order_id', how='left')

    df['total_amount'] = df['total_price'].fillna(0) + df['total_freight'].fillna(0)
    print('Built features from raw Olist files in', raw_dir)

# Preview
df.head(5)

Built features from raw Olist files in ..\..\brazilian dataset


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,items_count,total_price,total_freight,avg_item_price,payment_value,payment_installments,total_amount
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,29.99,8.72,29.99,38.71,1.0,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,118.70,22.76,118.70,141.46,1.0,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,159.90,19.22,159.90,179.12,3.0,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,45.00,27.20,45.00,72.20,1.0,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,19.90,8.72,19.90,28.62,1.0,28.62


In [5]:
# Feature engineering: select target and numeric features
import pandas as pd
from pandas.api.types import is_numeric_dtype

# Choose target column in priority order
target_candidates = ['total_amount', 'order_total_cents', 'order_total', 'payment_value']
target = next((c for c in target_candidates if c in df.columns), None)
if target is None:
    raise ValueError('Could not identify a target column in the dataset.')

# Target vector
y = pd.to_numeric(df[target], errors='coerce').fillna(0)

# Build feature matrix from numeric columns, excluding id/time/target
exclude = {target, 'order_id', 'customer_id', 'review_id'}
exclude |= {c for c in df.columns if 'date' in c or 'timestamp' in c}

num_cols = [c for c in df.columns if is_numeric_dtype(df[c]) and c not in exclude]

# If empty, coerce likely feature columns and retry
if not num_cols:
    for c in ['items_count', 'payment_installments', 'total_freight', 'avg_item_price', 'payment_value']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    num_cols = [c for c in ['items_count', 'payment_installments', 'total_freight', 'avg_item_price', 'payment_value'] if c in df.columns]

if not num_cols:
    raise ValueError('No numeric features available to train on.')

X = df[num_cols].fillna(0)

print('Target:', target)
print('Features:', num_cols[:10], f'... ({len(num_cols)} total)')

Target: total_amount
Features: ['items_count', 'total_price', 'total_freight', 'avg_item_price', 'payment_value', 'payment_installments'] ... (6 total)


In [6]:
# Train/test split, train model, evaluate, and save
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from pathlib import Path
import numpy as np
import joblib

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, preds)
print(f'RMSE: {rmse:.2f}, R2: {r2:.3f}')

# Save model
out_dir = Path('models')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'order_total_model.pkl'
joblib.dump(model, out_path)
print('Model saved to', out_path)

RMSE: 9.24, R2: 0.998
Model saved to models\order_total_model.pkl
Model saved to models\order_total_model.pkl
